# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [27]:
# Trying to first look at the data table by asking for head 
school_data_complete.head(5)

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget,overall_score,math_pass,reading_pass
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635,112.0,1,0
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635,108.0,0,1
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635,105.0,0,1
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635,91.5,0,0
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635,132.5,1,1


### Making functions to be used later

In [21]:
# Let's make a function based on the grade
def passing_grade(score):
    if score > 69:
        return 1
    else:
        return 0

### Getting the data and summary table

In [26]:
# Counta schools
counta_schools = school_data_complete["school_name"].count()

# Counta students 
counta_student = school_data_complete["student_name"].count()

# Sum of budget
sum_budget = school_data_complete["budget"].sum()

# Average math score
avg_math_score = school_data_complete["math_score"].mean()

# Calculate the average reading score
avg_reading_score = school_data_complete["reading_score"].mean()

# Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2
school_data_complete["overall_score"] = school_data_complete["math_score"]+school_data_complete["reading_score"]/2
overall_passing = school_data_complete[school_data_complete.overall_score > 69]
overall_pass = len(overall_passing)
overall_rate = overall_pass/counta_student*100

# Calculate the percentage of students with a passing math score (70 or greater)
school_data_complete["math_pass"] = school_data_complete["math_score"].apply(passing_grade)
math_pass = school_data_complete[school_data_complete.math_pass == True]
math_rate = math_pass["student_name"].count()/counta_student*100

# Calculate the percentage of students with a passing reading score (70 or greater)
school_data_complete["reading_pass"] = school_data_complete["reading_score"].apply(passing_grade)
reading_pass = school_data_complete[school_data_complete.reading_pass == True]
reading_rate = reading_pass["student_name"].count()/counta_student*100

# Create a dataframe to hold the above results
sum_tbl = {}
sum_df = pd.DataFrame(sum_tbl, index=None)
sum_df["Variables"] = ["Number of Schools", "Number of Students", "Total Budget", "Average Math Score",
                    "Average Reading Score", "Percent Pass (overall)", "Percent Pass (math)", 
                    "Percent Pass (reading)"]
sum_df["Values"] = [counta_schools, counta_student, sum_budget, avg_math_score, avg_reading_score, overall_rate,
                      math_rate, reading_rate]
sum_df

,Variables,Values
0,Number of Schools,3.917000e+04
1,Number of Students,3.917000e+04
2,Total Budget,8.293233e+10
3,Average Math Score,7.898537e+01
4,Average Reading Score,8.187784e+01
5,Percent Pass (overall),1.000000e+02
6,Percent Pass (math),7.498085e+01
7,Percent Pass (reading),8.580546e+01


## School Summary
* Create an overview table that summarizes key metrics about each school, including:
  * School Name ✓
  * School Type ✓
  * Total Students ✓
  * Total School Budget ✓
  * Per Student Budget ✓
  * Average Math Score ✓
  * Average Reading Score ✓
  * % Passing Math ✓
  * % Passing Reading ✓
  * Overall Passing Rate (Average of the above two) ✓
  
* Create a dataframe to hold the above results

In [130]:
# Getting the groupby table setup by school name and aggregate functions to be used later
base = pd.DataFrame(school_data_complete.groupby('school_name').agg({'type':'first',
                                                                      'student_name': 'count',
                                                                      'budget': 'sum',
                                                                      'math_score': 'mean',
                                                                      'reading_score': 'mean',
                                                                      'math_pass': 'mean',
                                                                      'reading_pass': 'mean'}))
# Calculating budget per student
base["Per Student Budget"] = (base["budget"]/base["student_name"])

# Calculating % Passing
base["math_pass"] = base["math_pass"]*100
base["reading_pass"] = base["reading_pass"]*100
base["Overall Passing Rate"] = (base["reading_pass"]+base["math_pass"])/2

base.columns = ['School Type','Total Students','Total School Budget','Average Math Score',
                'Average Reading Score','% Passing Math','% Passing Reading','Per Student Budget',
                'Overall Passing Rate']
base

,School Type,Total Students,Total School Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Per Student Budget,Overall Passing Rate
school_name,,,,,,,,,
Bailey High School,District,4976,15549641728,77.048432,81.033963,66.680064,81.933280,3124928.0,74.306672
Cabrera High School,Charter,1858,2009159448,83.061895,83.975780,94.133477,97.039828,1081356.0,95.586652
Figueroa High School,District,2949,5557128039,76.711767,81.158020,65.988471,80.739234,1884411.0,73.363852
Ford High School,District,2739,4831365924,77.102592,80.746258,68.309602,79.299014,1763916.0,73.804308
Griffin High School,Charter,1468,1346890000,83.351499,83.816757,93.392371,97.138965,917500.0,95.265668
Hernandez High School,District,4635,14007062700,77.289752,80.934412,66.752967,80.862999,3022020.0,73.807983
Holden High School,Charter,427,105933149,83.803279,83.814988,92.505855,96.252927,248087.0,94.379391
Huang High School,District,2917,5573322295,76.629414,81.182722,65.683922,81.316421,1910635.0,73.500171
Johnson High School,District,4761,14733628650,77.072464,80.966394,66.057551,81.222432,3094650.0,73.639992


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [53]:
top_by_pass_rate = base.nlargest(5, 'Overall Passing Rate')
top_by_pass_rate

,School Type,Total School Budget,Total Students,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Per Student Budget,Overall Passing Rate
school_name,,,,,,,,,
Cabrera High School,Charter,1858,2009159448,83.061895,83.975780,94.133477,97.039828,1081356.0,95.586652
Thomas High School,Charter,1635,1705517550,83.418349,83.848930,93.272171,97.308869,1043130.0,95.290520
Pena High School,Charter,962,563595396,83.839917,84.044699,94.594595,95.945946,585858.0,95.270270
Griffin High School,Charter,1468,1346890000,83.351499,83.816757,93.392371,97.138965,917500.0,95.265668
Wilson High School,Charter,2283,3012587442,83.274201,83.989488,93.867718,96.539641,1319574.0,95.203679


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [54]:
low_by_pass_rate = base.nsmallest(5, 'Overall Passing Rate')
low_by_pass_rate

,School Type,Total School Budget,Total Students,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Per Student Budget,Overall Passing Rate
school_name,,,,,,,,,
Rodriguez High School,District,3999,10186904637,76.842711,80.744686,66.366592,80.220055,2547363.0,73.293323
Figueroa High School,District,2949,5557128039,76.711767,81.158020,65.988471,80.739234,1884411.0,73.363852
Huang High School,District,2917,5573322295,76.629414,81.182722,65.683922,81.316421,1910635.0,73.500171
Johnson High School,District,4761,14733628650,77.072464,80.966394,66.057551,81.222432,3094650.0,73.639992
Ford High School,District,2739,4831365924,77.102592,80.746258,68.309602,79.299014,1763916.0,73.804308


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [112]:
by_grade = school_data_complete.groupby(['school_name','grade']).agg({'math_score':'mean'})
by_grade["school name"] = by_grade.index.get_level_values('school_name')
by_grade["grade"] = by_grade.index.get_level_values('grade')
by_grade.index.names = [' ',' ']
pivot_math = by_grade.pivot('school name', 'grade', 'math_score')
del pivot_math.index.name
del pivot_math.columns.name
pivot_math

,10th,11th,12th,9th
Bailey High School,76.996772,77.515588,76.492218,77.083676
Cabrera High School,83.154506,82.765560,83.277487,83.094697
Figueroa High School,76.539974,76.884344,77.151369,76.403037
Ford High School,77.672316,76.918058,76.179963,77.361345
Griffin High School,84.229064,83.842105,83.356164,82.044010
Hernandez High School,77.337408,77.136029,77.186567,77.438495
Holden High School,83.429825,85.000000,82.855422,83.787402
Huang High School,75.908735,76.446602,77.225641,77.027251
Johnson High School,76.691117,77.491653,76.863248,77.187857
Pena High School,83.372000,84.328125,84.121547,83.625455


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [114]:
by_grade = school_data_complete.groupby(['school_name','grade']).agg({'reading_score':'mean'})
by_grade["school name"] = by_grade.index.get_level_values('school_name')
by_grade["grade"] = by_grade.index.get_level_values('grade')
by_grade.index.names = [' ',' ']
pivot_reading = by_grade.pivot('school name', 'grade', 'reading_score')
del pivot_reading.index.name
del pivot_reading.columns.name
pivot_reading

,10th,11th,12th,9th
Bailey High School,80.907183,80.945643,80.912451,81.303155
Cabrera High School,84.253219,83.788382,84.287958,83.676136
Figueroa High School,81.408912,80.640339,81.384863,81.198598
Ford High School,81.262712,80.403642,80.662338,80.632653
Griffin High School,83.706897,84.288089,84.013699,83.369193
Hernandez High School,80.660147,81.396140,80.857143,80.866860
Holden High School,83.324561,83.815534,84.698795,83.677165
Huang High School,81.512386,81.417476,80.305983,81.290284
Johnson High School,80.773431,80.616027,81.227564,81.260714
Pena High School,83.612000,84.335938,84.591160,83.807273


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [152]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 2.5e+05, 7.5e+05, 1e+06, 2.5e+06, 5e+06]
group_names = ["<$250k", "$750k", "$1m", "$2.5m", ">$2.5m"]

In [153]:
base['binned'] = pd.cut(base['Per Student Budget'], bins=spending_bins, labels=group_names)
score_by_spending = base.groupby('binned').mean()
del score_by_spending.index.name
score_by_spending

,Total Students,Total School Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Per Student Budget,Overall Passing Rate
<$250k,427.00,1.059331e+08,83.803279,83.814988,92.505855,96.252927,248087.00,94.379391
$750k,962.00,5.635954e+08,83.839917,84.044699,94.594595,95.945946,585858.00,95.270270
$1m,1468.00,1.346890e+09,83.351499,83.816757,93.392371,97.138965,917500.00,95.265668
$2.5m,2242.75,3.304834e+09,80.904987,82.822740,83.556977,90.588593,1388627.75,87.072785
>$2.5m,4592.75,1.361931e+10,77.063340,80.919864,66.464293,81.059691,2947240.25,73.761992


## Scores by School Size

* Perform the same operations as above, based on school size.

In [157]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [159]:
base['binned_size'] = pd.cut(base['Total Students'], bins=size_bins, labels=group_names)
score_by_size = base.groupby('binned_size').mean()
del score_by_size.index.name
score_by_size

,Total Students,Total School Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Per Student Budget,Overall Passing Rate
Small (<1000),694.500,3.347643e+08,83.821598,83.929843,93.550225,96.099437,416972.500,94.824831
Medium (1000-2000),1704.400,1.762232e+09,83.374684,83.864438,93.599695,96.790680,1029597.200,95.195187
Large (2000-5000),3657.375,9.181455e+09,77.746417,81.344493,69.963361,82.766634,2333437.125,76.364998


## Scores by School Type

* Perform the same operations as above, based on school type.

In [160]:
binned_type = base.groupby('School Type').mean()
del binned_type.index.name
binned_type

,Total Students,Total School Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Per Student Budget,Overall Passing Rate
Charter,1524.250000,1.561659e+09,83.473852,83.896421,93.620830,96.586489,9.126881e+05,95.103660
District,3853.714286,1.006272e+10,76.956733,80.966636,66.548453,80.799062,2.478275e+06,73.673757
